# A Random Number Guessing Game

_Steve Taylor, September, 2021_

I've been asked to create a game in Python that creates a random number and makes you guess it.

A random number means I'll use the `random` package. Import it.

In [ ]:
%load_ext nb_black
import random

I've made a game where I am thinking of a number between 0 and 10. I'll tell you that.

In [ ]:
print("I've made a game where I'm thinking of a number between 0 and 10.")

I have to _store_ that number somewhere, so I call that piece of memory `random_number`. I'm going to print it so testing is easier, but I'll remove the `print()` statement when I bring my code to "production".

In [ ]:
# get a random integer between 0 and 10.
random_number = random.randint(0, 10)
print(random_number)

You only get three guesses. I should tell you _that_ too.

In [ ]:
print("You get three guesses.")

## Style 1

Okay! We're ready. I ask you for the first guess.

In [ ]:
first_guess = input("What is your first guess? ")
print(type(first_guess), first_guess)

Excellent guess I think. I know that when I ask you using `input()`, your response is a Python `str`, but I'll need to compare that to the `int` from `randint()` above. So I'll convert it.

In [ ]:
first_guess_int = int(first_guess)
print(type(first_guess_int), first_guess_int)

I will compare your answer to the random number. If you got it right, super. If not, we keep at it, eh?

In [ ]:
if first_guess_int == random_number:
    print("You got it in one try!")
else:
    print("Try 1. You did not guess it correctly. Guess again.")

Let's ask again. This feels repetative, but it's only the second time, so I'll suffer it.

In [ ]:
# I learned from the first time that I can call `int()` on the `input()` in one line... save some time.
second_guess = int(input("What is your second guess? "))

In [ ]:
if second_guess == random_number:
    print("You got it in the second try!")
else:
    print("Try 2. You did not guess it correctly. Guess again.")

Finally, a third time. I'm starting to wonder that what I'm building might not _scale_.

In [ ]:
# When I first do this I copy and paste, and I had "What is your second guess" instead of third. Sloppy, that.
third_guess = int(input("What is your third guess? "))
if third_guess == random_number:
    print("You got it in the third try!")
else:
    print(
        f"Try 3. You did not guess it correctly. Sorry! The number was {random_number}."
    )

As I play my game, trying different combinations, I find I can guess the number on the first try, but it still asks me for a second guess, and a third. Annoying!

I could add another variable that keeps track of which try is successful. E.g.,

```
if second_guess == random_number:
    print("You got it on the second try!")
    second_guess_success = True
    ...
```

In each subsequent check do things like,

```
if not second_guess_sucess and third_guess == random_number:
   ...
```

but that's going to get krufty fast. So what's next?

## Style 2

Same prompts: I've made a game, blah, blah. You get three guesses, blah, blah. 

I still need a random number. Might as well create a new one here.

In [ ]:
# get a random integer between 0 and 10.
random_number = random.randint(0, 10)  # Notice I'm reusing this variable.
print(random_number)

Same thing, ask the question, get an answer.

In [ ]:
first_guess = int(input("What is your first guess? "))
if first_guess == random_number:
    print("Success on the first try!")
else:
    print("No success on the first try. Try again.")
    second_guess = int(input("What is your second guess? "))
    if second_guess == random_number:
        print("Success on the second try!")
    else:
        print("No success on the second try. Try again.")
        third_guess = int(input("What is your third guess? "))
        if third_guess == random_number:
            print("Success on the third try!")
        else:
            print(f"No success on the third try. The number was {random_number}.")

That big chunk of goodness works. The copying and pasting took a while to sort out -- and I'm still not 100% sure I got everything. But it works! It's also monolithic and it doesn't _scale_. By that, what if I wanted to quickly make the game ask five times instead of three. Or even two instead of three. And like the first style it suffers from not being modular (read: functions!), and therefore it's hard to maintain.

## Style 3

I'm going to work with the second style, but use some functions to make some things reusable.

The first piece I want to reuse it the part where I tell you which try you're on, and ask for your guess.

A note on input validation. Testing that a human responds with what we've asked them for is a piece of work. Specifically, `int(input())` will fail if it gets anything that doesn't parse as an integer. And it will burn a guess if someone enters an 11 or -2343. Check out chapter 8 of the Sweigart text, or use your favourite search engine for looking for examples.

In [ ]:
def get_guess(try_number, lower=0, upper=10):
    """
    Mostly from Sweigart, chapter 8
    https://automatetheboringstuff.com/2e/chapter8/

    Prompts for an integer response between a specified range,
    given by lower and upper.

    Returns the guess as an int.
    """
    while True:
        guess = input(f"Guess #{try_number}. What is your guess? ")
        try:
            # attempt to cast/convert the str to an int
            guess = int(guess)
        except:
            # if what was in the try block failed, we'll end up here.
            print("Please use numeric digits.")
            # a hard bounce back to the top of the loop
            continue

        # we should have an integer now... let's check it for what we want
        if not lower <= guess <= upper:
            print(f"Please enter a number between {lower} and {upper}.")
            # bounce back to the top of the loop
            continue

        # we made it! we can return the guess and exit the function
        return guess


# get_guess(2)

I also note that I check the guess against the random number quite a lot. Maybe a good piece of reusable-ness?

In [ ]:
def check_guess(guess, random_number, try_number):
    """
    Check the guess against the random number,
    and print a status message.

    Return a True if the guess == random_number,
    else return False
    """
    if guess == random_number:
        print(f"SUCCESS on try #{try_number}!")
        return True
    else:
        print(f"No success on try #{try_number}.")
        return False


# I can test this cheaply,
# and that helps build my confidence to use it as a building-block in later code
assert check_guess(1, 1, 1), "1 should equal 1"
assert not check_guess(1, 2, 1), "1 does not equal 2"

I've noticed much later on in the notebook that I've typed the header comments maybe three or four or more times.

Way too many. As a rule, if I do something more than two times, I reduce it to a function.

In [ ]:
def print_header(number_of_guesses=3, lower=0, upper=10):
    """
    I originally had all of this hardcoded with 0 and 10, and you get 3 guesses, etc...
    but as I used and reused it in many different examples, it became more flexible
    and more reusable with parameters.
    """
    print(
        f"I've made a game where I'm thinking of a number between {lower} and {upper}."
    )
    print(f"You get {number_of_guesses} guesses.")


print_header()

So the chunk rewritten below is -- must be! -- the same as what we wrote in the second style example, but it's tightly wound now. It still suffers from scalability problems, but it's more modular now, yes? If I had to make this five tries or seven, it'd be mostly straight-forward to see it. Notice the 1, 2, 3-ness of it becomes easier to see?

In [ ]:
print_header()

random_number = random.randint(0, 10)
# print the random_number again to make it easier to test
print(random_number)

first_guess = get_guess(1)
if check_guess(first_guess, random_number, 1):
    # I don't want you to 'pass' this in this class
    pass
else:
    second_guess = get_guess(2)
    if check_guess(second_guess, random_number, 2):
        pass
    else:
        third_guess = get_guess(3)
        if check_guess(third_guess, random_number, 3):
            pass
        else:
            print(f"Sorry! The number was {random_number}.")

Same as above, but this version written without using `pass`. If your instinct here is that things are starting to focus up nicely, I'm with you.

In [ ]:
print_header()

random_number = random.randint(0, 10)
# print the random_number again to make it easier to test
print(random_number)

first_guess = get_guess(1)
if not check_guess(first_guess, random_number, 1):
    second_guess = get_guess(2)
    if not check_guess(second_guess, random_number, 2):
        third_guess = get_guess(3)
        if not check_guess(third_guess, random_number, 3):
            print(f"Sorry! The number was {random_number}.")

## Style 4

What makes me itch about style 3 is we can see a chunk for 1, then for 2, then for 3. That screams -- *screams* -- loop.

In [ ]:
print_header(3)

random_number = random.randint(0, 10)
# print the random_number again to make it easier to test
print(random_number)

# I'm going to use a while loop, because it's a game with human's involved,
# and human's are not always easy to set a 'for' loop to.

# We need to start with guess #1
count = 1

# assume we haven't won yet
win = False

# stay in loop while we're still guessing, all the way to 3
while count <= 3:

    # count is going to go 1, 2, 3 just like our code in style 3.
    first_guess = get_guess(count)
    if check_guess(first_guess, random_number, count):
        win = True
        print(f"You've won on try {count}.")

        # before we didn't have a way to drop out when we win. In a loop we do.
        break

    count += 1
    # end while

# we have to check to see if we've used up our guesses: if we haven't won, we've lost.
if not win:
    print(f"Sorry! The number was {random_number}.")

## Style 5

Now it's pretty tight. Time to wrap our code in a function.

Don't get hung up on the function definition using optional parameters. Easy!

In [ ]:
def numbersGame(number_of_guesses=3, debug=False):
    """
    Guess a number between 0 and 10.

    Parameters:
      - number_of_guesses, an int that sets the number of guesses in the game
      - debug: if True, print the random number for testing.
      
    Returns None on successful guess.
    """

    print_header(number_of_guesses)

    random_number = random.randint(0, 10)
    if debug:
        # print the random_number again to make it easier to test
        print(random_number)

    # I'm going to use a while loop, because it's a game with human's involved,
    # and human's are not always easy to set a count to.

    # We need to start with guess #1
    count = 1

    # stay in loop while we're still guessing, all the way to 3
    while count <= number_of_guesses:

        # count is going to go 1, 2, 3 just like our code in style 3.
        first_guess = get_guess(count)
        if check_guess(first_guess, random_number, count):

            # adding a human touch to output strings is its own art
            if count == 1:
                try_string = "try"
            else:
                try_string = "tries"
            print(f"Congrats! You guessed it in {count} {try_string}.")

            # If we've guessed right, we're DONE. Leave the function.
            return

        # trust the indent; we're still in the while loop here
        count += 1

    # We don't need to check the win here, because we wouldn't _get_ here otherwise
    print(f"Sorry! The number was {random_number}.")

In [ ]:
# all the same:
# numbersGame(3,False)
# numbersGame(number_of_guesses=3, debug=False) # named arguments are best
numbersGame()

## Coda

Showing the function with no comments. Don't let the size of a piece of code shake you. Note, I did pull out the try/tries formatting for clarity, but reflect on how that reads as a bit less friendly when playing the game. In style 5, lines 33-37, could you take that chunk and create a function for it?

In [ ]:
def numbersGame(number_of_guesses=3, debug=False):

    random_number = random.randint(0, 10)
    if debug:
        print(random_number)

    print_header(number_of_guesses)

    count = 1
    while count <= number_of_guesses:

        first_guess = get_guess(count)
        if check_guess(first_guess, random_number, count):
            print(f"Congrats! You guessed it on try {count}.")
            return

        count += 1

    print(f"Sorry! The number was {random_number}.")

In [ ]:
numbersGame()